In [4]:
import h5py
import os
import pandas as pd

# Vorbereitungen

## Dateien in Array schreiben

In [5]:
directory = 'datasetsRosen'
datasets = []

for filename in os.listdir(directory):
    file = h5py.File(directory + '/' + filename)
    if 'data' in file:
        datasets.append(file['data'])
    elif 'Daten' in file:
        datasets.append(file['Daten'])
        
print(f'{len(datasets)} Datensätze erfolgreich in datasets-Array eingelesen')

2000 Datensätze erfolgreich in datasets-Array eingelesen


## Dateien in Pandas Dataframe schreiben

In [6]:
def read_hdf5(file_path):
    """Read an HDF5 file into a Pandas DataFrame and return it along with attributes."""
    with h5py.File(file_path, 'r') as file:
        if 'data' in file:
            data_group_name = 'data'
        elif 'Daten' in file:
            data_group_name = 'Daten'
        else:
            print(f"'data' missing in: {file_path}")
            return None, None

        data_group = file[data_group_name]
        data_dict = {name: data_group[name][:] for name in data_group.keys()}

        # Extract attributes
        attributes = {attr: data_group.attrs[attr] for attr in data_group.attrs.keys()}
        data_id = attributes.get('id', 'Unknown')

        max_length = max(len(values) for values in data_dict.values())
        shorter_columns = {name: len(values) for name, values in data_dict.items() if len(values) < max_length}
        if shorter_columns:
            print(f"Inconsistency in: {data_id}")
            for name, length in shorter_columns.items():
                print(f" - Column name:: {name}, Length: {length}")
            return None, attributes

        data_frame = pd.DataFrame(data_dict)
        return data_frame, attributes

all_dataframes = {}
for file_name in os.listdir('datasetsRosen'):
    if file_name.endswith('.h5'):
        file_path = os.path.join('datasetsRosen', file_name)
        frame, frame_attrs = read_hdf5(file_path)
        if frame is not None:
            all_dataframes[file_name] = (frame, frame_attrs)
# print(all_dataframes)

Inconsistency in: 3a17c454-eb64-43b3-a280-bce5a667ba2c
 - Column name:: velocity, Length: 414
Inconsistency in: 60f01116-eb79-44de-b21f-78c573afee97
 - Column name:: velocity, Length: 867
Inconsistency in: 1f3b446d-1a7d-4b0c-bc7d-f6dfb2431fbe
 - Column name:: velocity, Length: 116
Inconsistency in: b10ec6d8-207b-44cf-9f96-e951799554ca
 - Column name:: velocity, Length: 693
Inconsistency in: 8a7c9d31-68db-4b5c-b57e-3dbfbbdbe345
 - Column name:: velocity, Length: 495
Inconsistency in: 34a95981-e1a9-469f-94ca-5a3d3d603bf6
 - Column name:: velocity, Length: 747
Inconsistency in: 0d2b40a2-2eba-4552-b42e-867554893ca6
 - Column name:: velocity, Length: 649
Inconsistency in: ae7ef2c0-fa00-4731-88f9-ef8e6b737529
 - Column name:: velocity, Length: 900
Inconsistency in: abbfa4f3-7c83-4177-a42e-5bc038014356
 - Column name:: velocity, Length: 258
Inconsistency in: f9a8dd0e-1eb7-46d7-8f26-0a4bfbb01792
 - Column name:: velocity, Length: 555
Inconsistency in: 73d87145-fb50-4d4d-8d51-0be647cb460e
 - Co

In [182]:
for name, (df, attrs) in all_dataframes.items():
    print(f"{name}: DataFrame Shape: {df.shape}, Attributes: {attrs}")
    print("First 5 lines of the columns:")
    print(df[['velocity', 'defect_channel', 'distance', 'magnetization', 'timestamp', 'wall_thickness']].head())
    print("\n")

d19dde88-83ba-45e6-b23a-e3f1412ffd55.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'America', 'id': 'd19dde88-83ba-45e6-b23a-e3f1412ffd55', 'instrument': 'Dog'}
First 5 lines of the columns:
   velocity  defect_channel  distance  magnetization              timestamp  \
0  0.542553             0.0  0.000000     -10.473481  b'1970-11-23T08:59:1'   
1  0.501728             0.0  0.217217      -7.340053  b'1970-11-23T09:05:3'   
2  0.408769             0.0  0.434434      -6.639923  b'1970-11-23T09:12:0'   
3  0.460477             0.0  0.651652      -9.356144  b'1970-11-23T09:18:2'   
4  0.787703             0.0  0.868869     -10.811810  b'1970-11-23T09:24:4'   

   wall_thickness  
0       20.124644  
1        7.955212  
2       12.045357  
3        8.114115  
4       21.662076  


8acbe2e4-b46c-40a2-b635-6df21f699d25.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'Africa', 'id': '8acbe2e4-b46c-40a2-b635-6df21f699d25', 'instrument': 'Elephant'}
First 5 lines o

KeyError: "['velocity'] not in index"

# Datenbereinigung !!ChatGPT und kacke!!

In [ ]:
import os
import h5py
import numpy as np

def check_file(filename):
    try:
        with h5py.File(filename, 'r') as file:
            # Überprüfe hier die Struktur und Integrität der Daten
            check_dataset(file, 'defect_channel')
            check_dataset(file, 'distance')
            check_dataset(file, 'magnetization')
            check_dataset(file, 'timestamp')
            check_dataset(file, 'velocity')
            check_dataset(file, 'wall_thickness')
    except Exception as e:
        print(f"Fehler beim Lesen von {filename}: {e}")

def check_dataset(file, dataset_name):
    try:
        dataset = file['data'][dataset_name][:]
        validate_numerical_data(dataset)
        check_missing_data(dataset)
        # Weitere spezifische Überprüfungen für jedes Dataset können hinzugefügt werden
    except KeyError:
        print(f"Fehler: Dataset '{dataset_name}' nicht vorhanden.")
    except Exception as e:
        print(f"Fehler bei der Überprüfung von '{dataset_name}': {e}")

def validate_numerical_data(dataset):
    # Beispiel: Überprüfe, ob alle Werte positiv sind
    if (dataset < 0).any():
        print("Fehler: Negative Werte in den Daten.")

# Weitere Validierungsbedingungen können hier hinzugefügt werden

def check_missing_data(dataset):
    # Beispiel: Überprüfe, ob NaN-Werte in den Daten vorhanden sind
    if np.isnan(dataset).any():
        print("Fehler: NaN-Werte in den Daten.")

# Weitere Überprüfungen auf fehlende Daten können hier hinzugefügt werden

def log_result(filename, result):
    with open("log.txt", "a") as log_file:
        log_file.write(f"{filename}: {result}\n")

# Iteriere über alle Dateien und prüfe jede einzelne
folder_path = "datasetsRosen"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".h5"):
        check_file(os.path.join(folder_path, file_name))


# Datenüberprüfung

## Welche Konfigurationen und Instrumente gibt es in den Datensätzen?

In [7]:
confs_directory = 'datasetsRosen'
configurations = []
instruments = []
configAmount = 0
instAmount = 0

for idx, filename in enumerate(os.listdir(confs_directory)):
    file = h5py.File(confs_directory + '/' + filename)
    if 'data' in file:
        configName = file['data'].attrs['configuration']
        if file['data'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['data'].attrs['instrument']
        if file['data'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)
    if 'Daten' in file:
        configName = file['Daten'].attrs['configuration']
        if file['Daten'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['Daten'].attrs['instrument']
        if file['Daten'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)

print(f'Menge an Configurations: {configAmount}')
print('Configurations: ', configurations)
print(f'Menge an Instruments: {instAmount}')
print('Instruments: ', instruments)

Menge an Configurations: 2000
Configurations:  ['America', 'Africa', 'Australia', 'Europe', 'Asia']
Menge an Instruments: 2000
Instruments:  ['Dog', 'Elephant', 'Pufferfish', 'Unicorn', 'Dolphin']


## Wie viele falsche / fehlende Datensätze gibt es?

Prüft auf: 
    Negative Werte,
    Fehlende Datensätze / Falsche Key Bezeichnung

In [175]:
# defect_channel, distance, magnetization, timestamp, velocity, wall_thickness
attribute = ("magnetization"
             "") # attribute ersetzen um verschiedene Attribute zu testen
count_negativeValue = 0
negativeValuesIdx = []
count_noDataset = 0
noDatasetIdx = []
eastereggCount = 0

for idx, dataset in enumerate(datasets):
    # print(dataset[attribute][...])
    if attribute in dataset:
        if b'Easteregg :)' not in dataset[attribute][...]:
            velocity_data = dataset[attribute][...].astype(float)
            if (velocity_data < 0).any():
                count_negativeValue += 1
                negativeValuesIdx.append(idx)
        else:
            eastereggCount += 1
    else:
        count_noDataset += 1
        noDatasetIdx.append(idx)
        
print(f"---------------------------------------------- {attribute} Analyse ----------------------------------------------")
print(f"Anzahl negative Werte: {count_negativeValue}\n Indizes: {negativeValuesIdx}\n")
print(f"Anzahl fehlender {attribute} Datensätze: {count_noDataset}\n Indizes: {noDatasetIdx}")
print("--------------------------------------------------------------------------------------------------------------")
print(f"Anzahl Eastereggs: {eastereggCount}")
print(datasets[1959].keys())

---------------------------------------------- magnetization Analyse ----------------------------------------------
Anzahl negative Werte: 965
 Indizes: [0, 3, 6, 10, 14, 18, 19, 21, 22, 25, 29, 32, 33, 34, 36, 38, 39, 45, 47, 48, 49, 51, 53, 54, 55, 57, 61, 63, 64, 67, 68, 69, 70, 72, 74, 81, 82, 89, 90, 95, 96, 97, 98, 104, 105, 108, 109, 112, 114, 116, 117, 119, 121, 124, 129, 130, 132, 135, 136, 141, 144, 145, 146, 152, 153, 158, 160, 161, 162, 166, 167, 169, 170, 173, 174, 176, 177, 178, 179, 181, 182, 184, 186, 187, 188, 192, 193, 194, 197, 198, 199, 202, 203, 205, 206, 207, 210, 214, 216, 218, 222, 225, 226, 227, 228, 230, 232, 235, 236, 240, 245, 248, 249, 257, 260, 261, 262, 264, 265, 266, 268, 269, 271, 272, 273, 275, 276, 278, 281, 282, 284, 285, 287, 288, 289, 291, 292, 293, 294, 295, 297, 299, 301, 302, 303, 305, 308, 310, 311, 315, 319, 322, 323, 324, 328, 329, 334, 335, 338, 340, 341, 342, 343, 346, 347, 348, 350, 351, 353, 354, 355, 356, 357, 359, 360, 361, 362, 363, 36